# How to translate MNE tables to API queries
This guide shows how to translate the process of getting data from MNE iTables searches to using the API using the Python Package.

## Setup

In [16]:
import beaapi

import pandas as pd
pd.set_option('display.max_rows', None)  # show all rows
pd.set_option('display.max_colwidth', None)  # show all text in cells

In [3]:
# Get key from unversioned file
from dotenv import dotenv_values
beakey = dotenv_values()["beakey"]

## Walk-through

Step 1a: Choose `DirectionofInvestment`. The first page of the iTables query asks you to pick the "Investment Type" which roughly maps to `DirectionOfInvestment`

* Choose “inward” for most “foreign direct investment in the United States” requests. However, for certain data (series ID = 8, 9) by U.S. state, choose “state”
* Choose “outward” for most “United States direct investment abroad” requests. However, choose “parent” for “U.S. Direct Investment Abroad / Data on activities of multinational enterprises / Nonbank U.S. Parent Companies (data up to 2008)” or “U.S. Direct Investment Abroad / Data on activities of multinational enterprises /All U.S. Parent Companies (data for 2009 and forward)”

Step 1b & 2: Choose a `SeriesID`

* The second question on first iTables page ask about the "Data Type" and this maps to a category of `SeriesID` (see below) that is then shown on the second iTables page (narrowed down given available `DirectionOfInvestment`).

DataType:

- 1-21 & 46-61: Data on activites of multinational enterprises (AMNE)
- 22-44: Balance of payments and direct investment position data
- 62-69: Data on new foreign direct investment in the U.S.

In [15]:
pd.read_csv("TA_MNE_series_list.csv").set_index('Series ID')

Direction  Note  \
Series ID                                      
0                                  NaN   NaN   
1                inward/outward/parent   NaN   
2                               inward   NaN   
3                       outward/parent   NaN   
4                inward/outward/parent   NaN   
5                inward/outward/parent   NaN   
6                inward/outward/parent   NaN   
7                inward/outward/parent   NaN   
8          inward/outward/state/parent   NaN   
9                                state   1.0   
10                      inward/outward   NaN   
11                             defunct   NaN   
12                      inward/outward   NaN   
13                             defunct   NaN   
14               inward/outward/parent   NaN   
16                             defunct   NaN   
19                             defunct   NaN   
20                              parent   NaN   
21                              parent   NaN   
22                              inward   NaN   
23                              inward   NaN   
24                              inward   NaN   
25                      inward/outward   NaN   
26                              inward   NaN   
27                      inward/outward   NaN   
28                              inward   2.0   
29                              inward   2.0   
30                             outward   NaN   
31                             outward   NaN   
32                             outward   NaN   
33                             outward   NaN   
34                             outward   2.0   
35                             outward   2.0   
39                             outward   2.0   
40                             outward   2.0   
41                              inward   2.0   
42                              inward   2.0   
43                              inward   2.0   
44                              inward   2.0   
46                              parent   NaN   
47                              parent   NaN   
48                              parent   NaN   
50                             defunct   NaN   
51                             defunct   NaN   
52                             defunct   NaN   
53                              inward   NaN   
54                             outward   NaN   
55                             defunct   NaN   
56                             defunct   NaN   
57                              parent   NaN   
58                             outward   NaN   
60                             outward   NaN   
61                             outward   NaN   
62                              inward   NaN   
64                              inward   NaN   
66                              inward   NaN   
68                              inward   NaN   
69                              inward   NaN   

                                             Short name  
Series ID                                                
0                                                   All  
1                                                Assets  
2                                             Gross PPE  
3                                  Capital Expenditures  
4                                                 Sales  
5                                            Net Income  
6                                           Value Added  
7                                          Compensation  
8                                            Employment  
9                              Manufacturing Employment  
10                                     Exports of Goods  
11                          Exports of Goods to the FPG  
12                                     Imports of Goods  
13                        Imports of Goods from the FPG  
14                                                  R&D  
16                                        Balance Sheet  
19                                  U.S. Trade in Goods  
20                                          Li

Notes:

1. Requires `classification="statebycountryofUBO"`, even though no country data are provided
2. Prior to 2006 only

Step 3: Choose `Classification`

- The classification depends on the direction and seriesID. The iTables are a guide as to which classifications are available
- "CountrybyIndustry" will yield a large number of countries broken down by major industry. "IndustrybyCountry" will yield the opposite.
- You can search for the iTables string in the table below

In [17]:
beaapi.get_parameter_values(beakey, 'MNE', 'Classification').set_index('key')

desc
key                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
Country                                                                                                                                                                                                                                                                                                                                        Set to classification equal to 'country' to obtain data for all countries for which data are available for the given series, ownership, and year.
CountryByDestination                                                                                                                                                                                                                                                                                                                                                         CountryByDestination returns values for country broken out by destination for goods supplied and services supplied.
CountryByIndustry                                                                                                                                                                                                                                                                                                                                                                                    CountryByIndustry provides data for a large set of countries broken down by major industry.
CountrybyType                                                                                                                                                                                                                                                                               Country by Type of Investment returns data on new foreign direct investment in the United States broken out by country of UBO and the type of investment (acquisition, establishment, or expansion).
CountryofUBO                     ? Presents data on foreign direct investment in the United States by the country of the Ultimate Beneficial Owner (UBO).  The ultimate beneficial owner (UBO) is the entity, proceeding up the foreign ownership chain, which is not more than 50 percent owned by another entity. The UBO is the entity that ultimately owns or controls an affiliated and thus ultimately derives the benefits and assumes the risks from owning or controlling an affiliate.
CountryofUBObyDestination                                                                                                                                                                                                                                                                    Presents data on services supplied by U.S. affiliates of foreign MNE?s, broken down by the country of Ultimate Beneficial Owner (UBO) and the destination (U.S. or foreign) of the service supplied
CountryofUBObyIndustry                                                                                                                                                                                                                                                                                                      Presents data on foreign direct investment in the United States broken down by the country of the Ultimate Beneficial Owner (UBO) and the industry of the affiliate.
countryofUBObyType                                                                                                                                 

Step 4: For AMNE estimates, Choose `Ownershiplevel` and `Nonbankaffiliatesonly`

- `Nonbankaffiliatesonly` should be set to zero for any data from after about 2009.
- More data are available for `ownershiplevel = "0"` (majority-owned affiliates only) than for `ownershiplevel = "1"` (all affiliates)

Step 5: Choose industry, country, year, etc…

- This is pretty straightforward. Full lists are show below. `Country = "all"` and `Industry = "all"` yield all results. We show full available options for both below.
- Years must be listed separately: `Year = "2010, 2011, 2012, 2013"` for example. `Year = "all"` returns all years


Countries:

In [15]:
display(beaapi.get_parameter_values(beakey, "MNE", parametername="Country").style.hide_index())

Industries:

In [20]:
industries = beaapi.get_parameter_values(beakey, "MNE", parametername="Industry")
display(industries.head(5).style.hide_index())

## Examples:

U.S. Direct investment abroad, total position, by industry, 2015-2020:

- U.S. Direct investment abroad requires directionofinvestment = outward
- Position is seriesID = 30
- Classification = industry
- Year = 2015,...,2020

In [6]:
beaapi.get_data(beakey, "MNE", DirectionOfInvestment="outward",
                Classification="Industry", SeriesID="30",
                Year="2015,2016,2017,2018,2019,2020", getfootnotes="yes").head(3)

Year  SeriesID                                         SeriesName  \
0  2015        30  U.S. Direct Investment Position Abroad on a Hi...   
1  2016        30  U.S. Direct Investment Position Abroad on a Hi...   
2  2018        30  U.S. Direct Investment Position Abroad on a Hi...   

                    Row ColumnGParent ColumnParent    Column RowCode  \
0  All Industries Total          None         None  Position    0000   
1  All Industries Total          None         None  Position    0000   
2  All Industries Total          None         None  Position    0000   

  ColumnCode ColumnParentCode ColumnGParentCode           TableScale  \
0        000                0                 0  Millions of Dollars   
1        000                0                 0  Millions of Dollars   
2        000                0                 0  Millions of Dollars   

  DataValueUnformatted TableColumnDisplayOrder TableRowDisplayOrder  DataValue  
0              5289071                    1.00                 1.00  5,289,071  
1              5518644                    1.00                 1.00  5,518,644  
2              5792290                    1.00                 1.00  5,792,290

Foreign direct investment in the United States, employment by majority-owned Canadian firms broken down by major industry, 2012 – 2019

- Foreign direct investment in the United States requires directionofinvestment = inward
- Employment series ID = 8
- Classification is CountrybyIndustry
- Majority-owned affiliates requires ownershiplevel = 0, Nonbankaffiliates = 0 for recent data
- Canada country code = 100
- Year = 2017, 2018, 2019

In [5]:
beaapi.get_data(beakey, "MNE", DirectionOfInvestment="inward",
                Classification="CountrybyIndustry", SeriesID="8", 
                Year="2012,2013,2014,2015,2016,2017,2018,2019", Country="100",
                getfootnotes="yes", Ownershiplevel="0", NonbankAffiliatesOnly="0").head(3)

Year  SeriesID  SeriesName     Row ColumnGParent ColumnParent  \
0  2012         8  Employment  Canada          None         None   
1  2013         8  Employment  Canada          None         None   
2  2015         8  Employment  Canada          None         None   

                 Column RowCode ColumnCode ColumnParentCode ColumnGParentCode  \
0  All Industries Total     100       0000                0                 0   
1  All Industries Total     100       0000                0                 0   
2  All Industries Total     100       0000                0                 0   

               TableScale DataValueUnformatted TableColumnDisplayOrder  \
0  Thousands of Employees                560.8                    1.00   
1  Thousands of Employees                580.9                    1.00   
2  Thousands of Employees                641.4                    1.00   

  TableRowDisplayOrder DataValue  
0                15.50     560.8  
1                15.50     580.9  
2                15.50     641.4

Foreign direct investment in the United States, employment of majority-owned affiliates, by state and country, 2018

- Foreign direct investment in the United States employment by state requires directionofinvestment = state
- Employment series ID = 8
- Classification is StatebyCountryofUBO
- Majority-owned affiliates requires ownershiplevel = 0, Nonbankaffiliates = 0 for recent data
- Year = 2018

In [4]:
beaapi.get_data(beakey, "MNE", DirectionOfInvestment="state", 
                OwnershipLevel="0", NonbankAffiliatesOnly="0",
                Classification="StatebyCountryofUBO", Year="2018", 
                Country="all", seriesID="8", GetFootnotes="yes").head(3)

Year  SeriesID  SeriesName                     Row ColumnGParent  \
0  2018         8  Employment  Total States and Areas          None   
1  2018         8  Employment  Total States and Areas          None   
2  2018         8  Employment  Total States and Areas        Europe   

  ColumnParent                   Column RowCode ColumnCode ColumnParentCode  \
0         None  --- All Countries Total      00        000                0   
1         None                   Canada      00        100                0   
2       Total                    Total       00        000              000   

  ColumnGParentCode              TableScale DataValueUnformatted  \
0                 0  Thousands of Employees               7862.2   
1                 0  Thousands of Employees                837.4   
2               399  Thousands of Employees               4923.2   

  TableColumnDisplayOrder TableRowDisplayOrder DataValue  
0                    1.00                 1.00   7,862.2  
1                    2.00                 1.00     837.4  
2                    3.00                 1.00   4,923.2

New foreign direct investment in the United States, Investment expenditures by type, by country, all years available

- New foreign direct investment in the United States requires directofinvestment = inward
- Series ID = 62
- Classification is CountryofUBObyType
- Year = all

In [3]:
beaapi.get_data(beakey, "MNE", DirectionOfInvestment="inward", 
                Classification="CountryofUBObyType", Year="all",
                seriesID="62", GetFootnotes="yes").head(3)

Year  SeriesID               SeriesName                  Row  \
0  2020        62  Investment Expenditures  All Countries Total   
1  2019        62  Investment Expenditures  All Countries Total   
2  2014        62  Investment Expenditures  All Countries Total   

             ColumnGParent ColumnParent  Column RowCode ColumnCode  \
0  First-year expenditures       Total   Total      000        000   
1  First-year expenditures       Total   Total      000        000   
2  First-year expenditures       Total   Total      000        000   

  ColumnParentCode ColumnGParentCode           TableScale  \
0              000               111  Millions of dollars   
1              000               111  Millions of dollars   
2              000               111  Millions of dollars   

  DataValueUnformatted TableColumnDisplayOrder TableRowDisplayOrder DataValue  
0               120674                    1.00                 1.00   120,674  
1               221215                    1.00                 1.00   221,215  
2               260487                    1.00                 1.00   260,487

U.S. parent companies, assets, by industry, 2018

- U.S. parent companies requires directionofinvestment = parent
- Series ID = 1
- Classification = industry
- U.S. parent data requires ownershiplevel = 1, Nonbankaffiliates = 0 for recent data
- Year = 2018

In [9]:
beaapi.get_data(beakey, "MNE", DirectionOfInvestment="parent", 
                Classification="Industry", SeriesID="1", Year="2018",
                getfootnotes="yes", Ownershiplevel="1", NonbankAffiliatesOnly="0").head(3)

Year  SeriesID    SeriesName                     Row ColumnGParent  \
0  2018         1  Total Assets    All Industries Total          None   
1  2018         1  Total Assets                  Mining          None   
2  2018         1  Total Assets  Oil and Gas Extraction          None   

  ColumnParent        Column RowCode ColumnCode ColumnParentCode  \
0         None  Total Assets    0000        180                0   
1         None  Total Assets    2198        180                0   
2         None  Total Assets    2111        180                0   

  ColumnGParentCode           TableScale DataValueUnformatted  \
0                 0  Millions of Dollars             43783215   
1                 0  Millions of Dollars               562877   
2                 0  Millions of Dollars               277707   

  TableColumnDisplayOrder TableRowDisplayOrder   DataValue  
0                    1.00                 1.00  43,783,215  
1                    1.00                16.50     562,877  
2                    1.00                17.50     277,707